### Manipulation of kynurenine PeaskDB de novo-assisted database search results of _T. weisflogii_ rot samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of kynurenine searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Multiple injections and fragmentation strategies included
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the running # (e.g., 233) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [2]:
cd /home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/kynurenine/MED_Weissrot_Fusion_UWPR2021_PEAKS_129_325/

/home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/kynurenine/MED_Weissrot_Fusion_UWPR2021_PEAKS_129_325


In [3]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [4]:
ls 

'325-T12dig-all_PEAKS129_DB search psm.csv'
 325-T12dig-all_PEAKS129_dno.csv
 325-T12dig-all_PEAKS129_peptide.csv
 325-T12dig-all_PEAKS129_protein-peptides.csv
 325-T12dig-all_PEAKS129_proteins.csv
 325-T12dig-all_PEAKS129_proteins.fasta


In [5]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup325 = pd.read_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/kynurenine/MED_Weissrot_Fusion_UWPR2021_PEAKS_129_325/325-T12dig-all_PEAKS129_peptide.csv")

# remove redundant rows
peaksdb325 = pd.DataFrame.drop_duplicates(peaksdbdup325)

print(peaksdb325.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb325.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb325['#Spec']
del peaksdb325['Accession']
del peaksdb325['PTM']
del peaksdb325['AScore']

mean_length = peaksdb325['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup325))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb325))

#look at the dataframe
peaksdb325.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area 325_T12_trypsin', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec 325_T12_trypsin', 'Accession', 'PTM', 'AScore'],
      dtype='object')
10.830882352941176
# redundant peaksdb peptides in combined dataframe 408
# nonredundant peaksdb peptides in combined dataframe 408


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,LPQVEGTGGDVQPSQDLVR,91.47,1994.0068,19,1.1,665.6769,73.36,206000000.0,14,16361,20210114_Weissrot_325_T12_trypsin_EThcD_120min...
1,VIGQNEAVDAVSNAIR,79.09,1654.8638,16,1.7,552.6295,89.47,298000000.0,14,20448,20210114_Weissrot_325_T12_trypsin_EThcD_120min...
2,AIDLIDEAASSIR,75.97,1372.7197,13,2.0,458.5815,106.62,265000000.0,14,24902,20210114_Weissrot_325_T12_trypsin_EThcD_120min...
3,DVPGTGNEFVGDFR,70.38,1508.6896,14,1.2,503.9044,87.47,81800000.0,14,19936,20210114_Weissrot_325_T12_trypsin_EThcD_120min...
4,RQTSELQHGR,67.51,1210.6167,10,1.1,404.5466,15.59,1250000.0,14,2338,20210114_Weissrot_325_T12_trypsin_EThcD_120min...


In [6]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb325['A'] = peaksdb325['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb325['C'] = peaksdb325['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb325['D'] = peaksdb325['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb325['E'] = peaksdb325['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb325['F'] = peaksdb325['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb325['G'] = peaksdb325['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb325['H'] = peaksdb325['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb325 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb325['I'] = peaksdb325['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb325['K'] = peaksdb325['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb325['L'] = peaksdb325['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb325['M'] = peaksdb325['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb325['N'] = peaksdb325['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb325['P'] = peaksdb325['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb325['Q'] = peaksdb325['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb325['R'] = peaksdb325['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb325['S'] = peaksdb325['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb325['T'] = peaksdb325['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb325['V'] = peaksdb325['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb325['W'] = peaksdb325['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb325['Y'] = peaksdb325['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb325['c-carb'] = peaksdb325['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb325['m-oxid'] = peaksdb325['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb325['n-deam'] = peaksdb325['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of acetylation of K's in each peptide
peaksdb325['q-deam'] = peaksdb325['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# use a count function to enumerate the # of acetylation of K's in each peptide
peaksdb325['x-kynu'] = peaksdb325['Peptide'].apply(lambda x: x.count('(+3.99)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb325['stripped_peptide'] = peaksdb325['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb325['stripped_length'] = peaksdb325['stripped_peptide'].apply(len)

peaksdb325['NAAF_num.'] = peaksdb325['Area'] / peaksdb325['stripped_length']

# total the number of modifications in sequence
peaksdb325['ptm-total'] = peaksdb325['c-carb'] + peaksdb325['m-oxid'] + peaksdb325['n-deam'] + \
peaksdb325['q-deam'] + peaksdb325['x-kynu']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb325['stripped_IL']= peaksdb325['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb325.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/kynurenine/325_T12_trypsin_combine_kynur_DB_FDR1.csv")

# check out the results
peaksdb325.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,c-carb,m-oxid,n-deam,q-deam,x-kynu,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,LPQVEGTGGDVQPSQDLVR,91.47,1994.0068,19,1.1,665.6769,73.36,206000000.0,14,16361,...,0,0,0,0,0,LPQVEGTGGDVQPSQDLVR,19,1.084211e+07,0,LPQVEGTGGDVQPSQDLVR
1,VIGQNEAVDAVSNAIR,79.09,1654.8638,16,1.7,552.6295,89.47,298000000.0,14,20448,...,0,0,0,0,0,VIGQNEAVDAVSNAIR,16,1.862500e+07,0,VLGQNEAVDAVSNALR
2,AIDLIDEAASSIR,75.97,1372.7197,13,2.0,458.5815,106.62,265000000.0,14,24902,...,0,0,0,0,0,AIDLIDEAASSIR,13,2.038462e+07,0,ALDLLDEAASSLR
3,DVPGTGNEFVGDFR,70.38,1508.6896,14,1.2,503.9044,87.47,81800000.0,14,19936,...,0,0,0,0,0,DVPGTGNEFVGDFR,14,5.842857e+06,0,DVPGTGNEFVGDFR
4,RQTSELQHGR,67.51,1210.6167,10,1.1,404.5466,15.59,1250000.0,14,2338,...,0,0,0,0,0,RQTSELQHGR,10,1.250000e+05,0,RQTSELQHGR


In [7]:
# keep only stripped peptide I/L and NAAF
db_325 = peaksdb325[['stripped_IL', 'Area', 'NAAF_num.']]

db_325.set_index('stripped_IL')

# write modified dataframe to new txt file
db_325.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/kynurenine/I-L_NAAFs/T12_325_trypsin_DB_kynur_peptides.csv")

db_325.head()

,stripped_IL,Area,NAAF_num.
0,LPQVEGTGGDVQPSQDLVR,206000000.0,1.084211e+07
1,VLGQNEAVDAVSNALR,298000000.0,1.862500e+07
2,ALDLLDEAASSLR,265000000.0,2.038462e+07
3,DVPGTGNEFVGDFR,81800000.0,5.842857e+06
4,RQTSELQHGR,1250000.0,1.250000e+05


### Exporting txt files of stripped peptides at confidence cutoffs:

In [8]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep325moddup = peaksdb325[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep325dup = peaksdb325[["stripped_peptide"]]

# deduplicate both of these lists
pep325mod = pep325moddup.drop_duplicates()
pep325 = pep325dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 325:", len(pep325moddup))
print("Deduplicated modified PeaksDB peptides in 325:", len(pep325mod))
print("Total stripped PeaksDB peptides in 325:", len(pep325dup))
print("Deduplicated stripped PeaksDB peptides in 325:", len(pep325))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep325.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/kynurenine/325_T12_trypsin_combine_kynur_DB_FDR1_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/rot-mayer/data/processed/kynurenine/325_T12_trypsin_combine_kynur_DB_FDR1_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/rot-mayer/data/processed/kynurenine/325_T12_trypsin_combine_kynur_DB_FDR1_stripped_peptides.fas

# look at the stripped peptides
pep325.head()

Total modified PeaksDB peptides in 325: 408
Deduplicated modified PeaksDB peptides in 325: 408
Total stripped PeaksDB peptides in 325: 408
Deduplicated stripped PeaksDB peptides in 325: 389


,stripped_peptide
0,LPQVEGTGGDVQPSQDLVR
1,VIGQNEAVDAVSNAIR
2,AIDLIDEAASSIR
3,DVPGTGNEFVGDFR
4,RQTSELQHGR


## NAAF correction and exporting files with AA and PTM totals:

In [9]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb325['A'].sum(),
        'C': peaksdb325['C'].sum(),
        'D': peaksdb325['D'].sum(),
        'E': peaksdb325['E'].sum(),
        'F': peaksdb325['F'].sum(),
        'G': peaksdb325['G'].sum(),
        'H': peaksdb325['H'].sum(),
        'I': peaksdb325['I'].sum(),
        'K': peaksdb325['K'].sum(),
        'L': peaksdb325['L'].sum(),
        'M': peaksdb325['M'].sum(),
        'N': peaksdb325['N'].sum(),
        'P': peaksdb325['P'].sum(),
        'Q': peaksdb325['Q'].sum(),
        'R': peaksdb325['R'].sum(),
        'S': peaksdb325['S'].sum(),
        'T': peaksdb325['T'].sum(),
        'V': peaksdb325['V'].sum(),
        'W': peaksdb325['W'].sum(),
        'Y': peaksdb325['Y'].sum(),
        'c-carb': peaksdb325['c-carb'].sum(),
        'm-oxid': peaksdb325['m-oxid'].sum(),
        'n-deam': peaksdb325['n-deam'].sum(),
        'q-deam': peaksdb325['q-deam'].sum(),
        'x-kynu': peaksdb325['x-kynu'].sum(),
        'Total area': peaksdb325['Area'].sum(),
        'Total length': peaksdb325['stripped_length'].sum()
       }

totalpeaksdb325 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'x-kynu'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb325['% C w/ carb'] = totalpeaksdb325['c-carb'] / totalpeaksdb325['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb325['% M w/ oxid'] = totalpeaksdb325['m-oxid'] / totalpeaksdb325['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb325['% N w/ deam'] = totalpeaksdb325['n-deam'] / totalpeaksdb325['N'] 

# calculate percentage of N's that are deamidated
totalpeaksdb325['% Q w/ deam'] = totalpeaksdb325['q-deam'] / totalpeaksdb325['Q'] 

# calculate percentage of X's that are deamidated
totalpeaksdb325['% X w/ kyur'] = totalpeaksdb325['x-kynu'] / totalpeaksdb325['N'] 

# calculate NAAF denominator for all peptides in dataset i
#totalpeaksdb325['NAAF denom.'] = peaksdb325['Total area'] / peaksdb325['Total length']

# write modified dataframe to new txt file
totalpeaksdb325.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/kynurenine/325_T12_trypsin_combine_kynur_DB_FDR1_totals.csv")

totalpeaksdb325.head()

,A,C,D,E,F,G,H,I,K,L,...,c-carb,m-oxid,n-deam,q-deam,x-kynu,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,% X w/ kyur
sample total,342,33,350,281,212,561,94,295,174,346,...,33,11,11,10,0,1.0,0.34375,0.047009,0.06135,0.0


In [ ]:
# use the calculated NAAF factor (in totalpeaksdb325 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 555224.331024

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb325['NAAF factor'] = (peaksdb325['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksdb325_NAAF = peaksdb325[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'k-oxid', 'p-oxid', 'r-oxid', 'y-oxid', 'n-deam', \
                              'k-meth', 'r-meth', 'q-pyro', 'k-acet']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb325_NAAF['A-NAAF20'] = peaksdb325_NAAF['A'] * peaksdb325['NAAF factor']
peaksdb325_NAAF['C-NAAF20'] = peaksdb325_NAAF['C'] * peaksdb325['NAAF factor']
peaksdb325_NAAF['D-NAAF20'] = peaksdb325_NAAF['D'] * peaksdb325['NAAF factor']
peaksdb325_NAAF['E-NAAF20'] = peaksdb325_NAAF['E'] * peaksdb325['NAAF factor']
peaksdb325_NAAF['F-NAAF20'] = peaksdb325_NAAF['F'] * peaksdb325['NAAF factor']
peaksdb325_NAAF['G-NAAF20'] = peaksdb325_NAAF['G'] * peaksdb325['NAAF factor']
peaksdb325_NAAF['H-NAAF20'] = peaksdb325_NAAF['H'] * peaksdb325['NAAF factor']
peaksdb325_NAAF['I-NAAF20'] = peaksdb325_NAAF['I'] * peaksdb325['NAAF factor']
peaksdb325_NAAF['K-NAAF20'] = peaksdb325_NAAF['K'] * peaksdb325['NAAF factor']
peaksdb325_NAAF['L-NAAF20'] = peaksdb325_NAAF['L'] * peaksdb325['NAAF factor']
peaksdb325_NAAF['M-NAAF20'] = peaksdb325_NAAF['M'] * peaksdb325['NAAF factor']
peaksdb325_NAAF['N-NAAF20'] = peaksdb325_NAAF['N'] * peaksdb325['NAAF factor']
peaksdb325_NAAF['P-NAAF20'] = peaksdb325_NAAF['P'] * peaksdb325['NAAF factor']
peaksdb325_NAAF['Q-NAAF20'] = peaksdb325_NAAF['Q'] * peaksdb325['NAAF factor']
peaksdb325_NAAF['R-NAAF20'] = peaksdb325_NAAF['R'] * peaksdb325['NAAF factor']
peaksdb325_NAAF['S-NAAF20'] = peaksdb325_NAAF['S'] * peaksdb325['NAAF factor']
peaksdb325_NAAF['T-NAAF20'] = peaksdb325_NAAF['T'] * peaksdb325['NAAF factor']
peaksdb325_NAAF['V-NAAF20'] = peaksdb325_NAAF['V'] * peaksdb325['NAAF factor']
peaksdb325_NAAF['W-NAAF20'] = peaksdb325_NAAF['W'] * peaksdb325['NAAF factor']
peaksdb325_NAAF['Y-NAAF20'] = peaksdb325_NAAF['Y'] * peaksdb325['NAAF factor']

# multiply the NAAF20 factor by the PTMs normalize its abundance by peak area and peptide length

peaksdb325_NAAF['ccarb-NAAF20'] = peaksdb325_NAAF['c-carb'] * peaksdb325_NAAF['NAAF factor']
peaksdb325_NAAF['moxid-NAAF20'] = peaksdb325_NAAF['m-oxid'] * peaksdb325_NAAF['NAAF factor']
peaksdb325_NAAF['koxid-NAAF20'] = peaksdb325_NAAF['k-oxid'] * peaksdb325_NAAF['NAAF factor']
peaksdb325_NAAF['poxid-NAAF20'] = peaksdb325_NAAF['p-oxid'] * peaksdb325_NAAF['NAAF factor']
peaksdb325_NAAF['roxid-NAAF20'] = peaksdb325_NAAF['r-oxid'] * peaksdb325_NAAF['NAAF factor']
peaksdb325_NAAF['yoxid-NAAF20'] = peaksdb325_NAAF['y-oxid'] * peaksdb325_NAAF['NAAF factor']
peaksdb325_NAAF['ndeam-NAAF20'] = peaksdb325_NAAF['n-deam'] * peaksdb325_NAAF['NAAF factor']
peaksdb325_NAAF['kmeth-NAAF20'] = peaksdb325_NAAF['k-meth'] * peaksdb325_NAAF['NAAF factor']
peaksdb325_NAAF['rmeth-NAAF20'] = peaksdb325_NAAF['r-meth'] * peaksdb325_NAAF['NAAF factor']
peaksdb325_NAAF['qpyro-NAAF20'] = peaksdb325_NAAF['q-pyro'] * peaksdb325_NAAF['NAAF factor']
peaksdb325_NAAF['kacet-NAAF20'] = peaksdb325_NAAF['k-acet'] * peaksdb325_NAAF['NAAF factor']

# write the dataframe to a new csv
peaksdb325_NAAF.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_325_T12_trypsin_combine_PTMopt_DB_FDR1_NAAF.csv")

peaksdb325_NAAF.head()

In [ ]:
# made a new dataframe that contains the sums of NAAF normalized AAs for peaksdb325 peaksdb results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb325_NAAF['NAAF factor'].sum(),
        'A': peaksdb325_NAAF['A-NAAF20'].sum(),
        'C': peaksdb325_NAAF['C-NAAF20'].sum(),
        'D': peaksdb325_NAAF['D-NAAF20'].sum(),
        'E': peaksdb325_NAAF['E-NAAF20'].sum(),
        'F': peaksdb325_NAAF['F-NAAF20'].sum(),
        'G': peaksdb325_NAAF['G-NAAF20'].sum(),
        'H': peaksdb325_NAAF['H-NAAF20'].sum(),
        'I': peaksdb325_NAAF['I-NAAF20'].sum(),
        'K': peaksdb325_NAAF['K-NAAF20'].sum(),
        'L': peaksdb325_NAAF['L-NAAF20'].sum(),
        'M': peaksdb325_NAAF['M-NAAF20'].sum(),
        'N': peaksdb325_NAAF['N-NAAF20'].sum(),
        'P': peaksdb325_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb325_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb325_NAAF['R-NAAF20'].sum(),
        'S': peaksdb325_NAAF['S-NAAF20'].sum(),
        'T': peaksdb325_NAAF['T-NAAF20'].sum(),
        'V': peaksdb325_NAAF['V-NAAF20'].sum(),
        'W': peaksdb325_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb325_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb325_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb325_NAAF['moxid-NAAF20'].sum(),
        'k-oxid': peaksdb325_NAAF['koxid-NAAF20'].sum(),
        'p-oxid': peaksdb325_NAAF['poxid-NAAF20'].sum(),
        'r-oxid': peaksdb325_NAAF['roxid-NAAF20'].sum(),
        'y-oxid': peaksdb325_NAAF['yoxid-NAAF20'].sum(),
        'n-deam': peaksdb325_NAAF['ndeam-NAAF20'].sum(),
        'k-meth': peaksdb325_NAAF['kmeth-NAAF20'].sum(),
        'r-meth': peaksdb325_NAAF['rmeth-NAAF20'].sum(),
        'q-pyro': peaksdb325_NAAF['qpyro-NAAF20'].sum(),
        'k-acet': peaksdb325_NAAF['kacet-NAAF20'].sum()
       }

totalpeaksdb325_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'k-oxid', 'p-oxid', 'r-oxid', \
                                                   'y-oxid', 'n-deam', 'k-meth', 'r-meth', \
                                                   'q-pyro', \
                                                   'k-acet'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb325_NAAF['% C w/ carb.'] = totalpeaksdb325_NAAF['c-carb'] / totalpeaksdb325_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb325_NAAF['% M w/ oxid'] = totalpeaksdb325_NAAF['m-oxid'] / totalpeaksdb325_NAAF['M'] 

# calculate NAAF-corrected percentage of K's that are oxidized
totalpeaksdb325_NAAF['% K w/ oxid'] = totalpeaksdb325_NAAF['k-oxid'] / totalpeaksdb325_NAAF['K'] 

# calculate NAAF-corrected percentage of P's that are oxidized
totalpeaksdb325_NAAF['% P w/ oxid'] = totalpeaksdb325_NAAF['p-oxid'] / totalpeaksdb325_NAAF['P'] 

# calculate NAAF-corrected percentage of R's that are oxidized
totalpeaksdb325_NAAF['% R w/ oxid'] = totalpeaksdb325_NAAF['r-oxid'] / totalpeaksdb325_NAAF['R'] 

# calculate NAAF-corrected percentage of Y's that are oxidized
totalpeaksdb325_NAAF['% Y w/ oxid'] = totalpeaksdb325_NAAF['y-oxid'] / totalpeaksdb325_NAAF['Y'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb325_NAAF['% N w/ deam'] = totalpeaksdb325_NAAF['n-deam'] / totalpeaksdb325_NAAF['N'] 

# calculate NAAF-corrected percentage of K's that are methylated
totalpeaksdb325_NAAF['% K w/ meth'] = totalpeaksdb325_NAAF['k-meth'] / totalpeaksdb325_NAAF['K'] 

# calculate NAAF-corrected percentage of R's that are methylated
totalpeaksdb325_NAAF['% R w/ meth'] = totalpeaksdb325_NAAF['r-meth'] / totalpeaksdb325_NAAF['R'] 

# calculate NAAF-corrected percentage of Q's that are pyro glu'd
totalpeaksdb325_NAAF['% Q w/ pyro'] = totalpeaksdb325_NAAF['q-pyro'] / totalpeaksdb325_NAAF['Q'] 

# calculate NAAF-corrected percentage of K's that are methylated
totalpeaksdb325_NAAF['% K w/ acet'] = totalpeaksdb325_NAAF['k-acet'] / totalpeaksdb325_NAAF['K'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb325_NAAF['NAAF check'] = totalpeaksdb325_NAAF['NAAF'] / 555224.331024

# write modified dataframe to new txt file, same name + totals
totalpeaksdb325_NAAF.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_325_T12_trypsin_combine_PTMopt_DB_FDR1_NAAF_totals.csv")

totalpeaksdb325_NAAF.head()